# Advanced Tutorial on MetaSynth

In this tutorial, we will be creating a `generative metadata format` (`gmf`) metadata file from a dataset using MetaSynth. We are going to walk through some of the advanced abilities of MetaSynth, such as handling dates, setting distributions and ensuring uniqueness in columns. This example workflow starts from a `.csv` file as input, but it easily adapted to other formats. 

You can run this notebook by checking out the MetaSynth repo and installing metasynth with `pip install metasynth`

In [1]:
# %pip install metasynth

In [2]:
# import required packages
import datetime as dt
import pandas as pd
from metasynth import MetaDataset
from utils import get_demonstration_fp

## Step 1: Transforming your data into a pandas DataFrame

The first step in creating the metadata is reading and converting your dataset to a pandas DataFrame. 

In [3]:
demonstration_fp = get_demonstration_fp()
df = pd.read_csv(demonstration_fp)
df.head()

,PassengerId,Name,Sex,Age,Parch,Fare,Cabin,Embarked,Birthday,Board time,Married since,all_NA
0,1,"Braund, Mr. Owen Harris",male,22.0,0,7.2500,NaN,S,1937-06-07,13:17:47,2022-07-21 17:34:14,NaN
1,2,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,0,71.2833,C85,C,1917-04-06,16:10:59,2022-08-09 21:00:09,NaN
2,3,"Heikkinen, Miss. Laina",female,26.0,0,7.9250,NaN,S,1910-09-13,17:46:26,2022-08-07 22:47:31,NaN
3,4,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,0,53.1000,C123,S,1938-08-18,10:52:32,2022-08-03 08:37:45,NaN
4,5,"Allen, Mr. William Henry",male,35.0,0,8.0500,NaN,S,1932-02-09,15:38:14,2022-08-02 08:46:23,NaN


MetaSynth will automatically generate the metadata from this DataFrame object so it is important to __ensure the data types for all the variables are correct__. For example, in the dataset above we see that Age is a floating point number whereas it should be an integer (22 instead of 22.0). In addition, there are a few categorical variables (Sex, Parch, Embarked) which are loaded in as string data types.

In general, we support [pandas dtypes](https://pandas.pydata.org/pandas-docs/stable/user_guide/basics.html#basics-dtypes). For our example dataset we can specify the `dtypes` and load the dataset as follows:

In [4]:
dtypes = {
    "Survived": "category",  # Categories should be assigned this type.
    "Name": "string",  # Strings should be assigned like this
    "Age": "Int64",  # Integer columns that have NA's in them should be explicitly nullable integers.
    "Sex": "category",
    "SibSp": "category",
    "Parch": "category",
    "Ticket": "string",
    "Cabin": "string",
    "Embarked": "category",
}
df = pd.read_csv(demonstration_fp, dtype=dtypes, parse_dates=["Married since"])  # Parse datetimes with parse_dates
df.head()

,PassengerId,Name,Sex,Age,Parch,Fare,Cabin,Embarked,Birthday,Board time,Married since,all_NA
0,1,"Braund, Mr. Owen Harris",male,22,0,7.2500,<NA>,S,1937-06-07,13:17:47,2022-07-21 17:34:14,NaN
1,2,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38,0,71.2833,C85,C,1917-04-06,16:10:59,2022-08-09 21:00:09,NaN
2,3,"Heikkinen, Miss. Laina",female,26,0,7.9250,<NA>,S,1910-09-13,17:46:26,2022-08-07 22:47:31,NaN
3,4,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35,0,53.1000,C123,S,1938-08-18,10:52:32,2022-08-03 08:37:45,NaN
4,5,"Allen, Mr. William Henry",male,35,0,8.0500,<NA>,S,1932-02-09,15:38:14,2022-08-02 08:46:23,NaN


### Dates and times

While pandas can easily parse datetime columns, it cannot parse proper times and dates. Instead, we use the types from the built-in `datetime` package. Thus we have to manually transform the strings in the columns with date, time, and datetimes to their proper objects. Since the columns in our example dataset follow the standard ISO-format, we can convert them with the `fromisoformat` method. If they are written in a different format, check out the [datetime library documentation](https://docs.python.org/3/library/datetime.html) on how to convert the strings to datetime/time/date objects.

Note the check for NA dates/times, otherwise it will raise an error instead.

In [5]:
df["Birthday"] = [dt.date.fromisoformat(x) if not pd.isna(x) else pd.NA for x in df["Birthday"]]
df["Board time"] = [dt.time.fromisoformat(x) if not pd.isna(x) else pd.NA for x in df["Board time"]]

Now, let's check the data types of our DataFrame:

In [6]:
df.dtypes

PassengerId               int64
Name                     string
Sex                    category
Age                       Int64
Parch                  category
Fare                    float64
Cabin                    string
Embarked               category
Birthday                 object
Board time               object
Married since    datetime64[ns]
all_NA                  float64
dtype: object

We see that most variables are now nicely specified as strings, categories and ints where necessary. For the dates and times we just created, we see the dtype `object`. This is the "catch-all" dtype for pandas. But don't worry, these columns have the correct type and MetaSynth will deal with it correctly:

In [7]:
df["Birthday"][0]

datetime.date(1937, 6, 7)

## Step 2: Creating a MetaDataset object from a DataFrame

Now a lot of work has already gone into creating a properly formatted dataframe. This work pays off at this stage: let's convert the DataFrame to a meta_dataset structure with the default options. Note: this takes a little bit of time!

In [8]:
meta_dataset = MetaDataset.from_dataframe(df)

/home/qubix/Documents/shared_work/synthetic/meta-synth/metasynth/disttree.py:127: UserWarning: 
Variable PassengerId seems unique, but not set to be unique.
Set the variable to be either unique or not unique to remove this warning.

  warnings.warn(f"\nVariable {series.name} seems unique, but not set to be unique.\n"


Then, we can show the metadata as a dictionary:

In [9]:
print(meta_dataset)

# Rows: 891
# Columns: 12

{'name': 'PassengerId', 'description': None, 'type': 'discrete', 'dtype': 'int64', 'prop_missing': 0.0, 'distribution': "{'name': 'DiscreteUniformDistribution', 'parameters': {'low': 1, 'high': 892}}"}

{'name': 'Name', 'description': None, 'type': 'string', 'dtype': 'string', 'prop_missing': 0.0, 'distribution': '.[]{12,82}'}

{'name': 'Sex', 'description': None, 'type': 'categorical', 'dtype': 'category', 'prop_missing': 0.0, 'distribution': "{'name': 'MultinoulliDistribution', 'parameters': {'labels': array(['female', 'male'], dtype='<U6'), 'probs': array([0.35241302, 0.64758698])}}"}

{'name': 'Age', 'description': None, 'type': 'discrete', 'dtype': 'Int64', 'prop_missing': 0.19865319865319866, 'distribution': "{'name': 'DiscreteUniformDistribution', 'parameters': {'low': 0, 'high': 81}}"}

{'name': 'Parch', 'description': None, 'type': 'categorical', 'dtype': 'category', 'prop_missing': 0.0, 'distribution': "{'name': 'MultinoulliDistribution', 'parameter

Note that the column with all NA's has been converted to an exponential distribution. This has no effect on the outcome, since it will generate NA's exclusively.

## Step 3: Saving the metadata in a file

After creating the metadata, we can save it to a file. The default format is `json`, meaning the file is quite legible by humans and computers alike. Therefore, it can be checked by the data controller and, when the disclosure risk is deemed to be low, this file can be shared with others.

In [10]:
file_path = "demonstration_metadata.json"
meta_dataset.to_json(file_path)

## Step 4: Generating synthetic data from the metadata

Upon receiving this file, you can use the MetaSynth package to generate a synthetic version of the dataset:

In [11]:
new_meta_dataset = MetaDataset.from_json(file_path)
new_meta_dataset.synthesize(5)

,PassengerId,Name,Sex,Age,Parch,Fare,Cabin,Embarked,Birthday,Board time,Married since,all_NA
0,424,>2.hi|cU: j k9oZ1-7dWgz0s#b`vA,female,42,0,63.887062,<NA>,S,1926-05-10,14:13:45,2022-07-25 13:56:22,NaN
1,7,RwD/c{$*Apox8dHbL wrcFq/V`Q3?rH/'2MstOyam ;3 ...,male,66,0,6.388892,<NA>,S,1926-01-11,17:27:10,2022-07-31 04:09:29,NaN
2,484,"dJ8K,#t""RX4:jZ,4+o+&idAN vY|-az!Sx.5[y> bM+jn|...",male,<NA>,0,9.740988,<NA>,S,1918-09-10,13:14:59,2022-07-22 13:01:32,NaN
3,338,"IWC ~8Z ew&,",female,<NA>,0,61.346274,<NA>,S,1932-07-09,None,NaT,NaN
4,200,VmbXgYR+1 K5d9w!{85InbQcH*+<KvUZ2Z]|_1$Zs|=F @...,male,44,0,5.999618,A@F9SCh,S,1936-10-10,12:38:58,2022-08-10 10:30:14,NaN


As you can see, the fake data looks a lot like the real data! However, it could still use some improvement. In the next sections, we will explore manual changes we can make to improve the quality of the synthetic data.

## Step 5: Improving the quality of the synthetic data

### Set unique columns

One column (PassengerId) has been detected as possibly unique by MetaSynth, as indicated by the following warning:

> "Variable PassengerId seems unique, but not set to be unique."

This column holds a variable with unique passenger identifiers, so in fact we do want synthetic data generated for this column to be unique as well. We can add this to the metadata by creating a list of options which we call a `specification`, or `spec`:

In [12]:
# First, we create a specification dictionary for the variables
var_spec = {
    "PassengerId": {"unique": True}
}

# then, we add that dictionary as the `spec` argument
meta_dataset = MetaDataset.from_dataframe(df, spec=var_spec)

# then, let's check what the metadata about PassengerId contains!
meta_dataset["PassengerId"].to_dict()

{'name': 'PassengerId',
 'type': 'discrete',
 'dtype': 'int64',
 'prop_missing': 0.0,
 'distribution': {'name': 'UniqueKeyDistribution',
  'parameters': {'low': 1, 'consecutive': 1}}}

So let's check what is generated from this new metadata:

In [13]:
meta_dataset.synthesize(5)

,PassengerId,Name,Sex,Age,Parch,Fare,Cabin,Embarked,Birthday,Board time,Married since,all_NA
0,1,V!&iJZ=8!VPz]-X' /W!]&-BU}IT,female,60,0,16.690308,<NA>,S,1930-04-13,15:01:19,NaT,NaN
1,2,_nh97JFPnox)! &E)<$|.RDjf,male,42,1,33.389448,FCC`xL@mq^pn'R,S,1929-08-28,16:48:02,2022-08-06 15:35:54,NaN
2,3,".Ag\zz=@j;n,4=^mUQE r8Qm}=eBcrfBeO}zU&e< S@Ih...",male,76,0,7.989553,"CXM""biwlA""t\k.",S,1912-02-20,15:56:36,2022-08-06 07:25:20,NaN
3,4,Sk0oBAil6CND=_>[ZkB7,male,68,2,25.659936,<NA>,C,1935-03-23,16:48:44,2022-08-02 11:22:26,NaN
4,5,#&P MV53${>|~6Ed;rKW)Fo<o_*| e$kkb 1X$7i}z)5T=...,male,35,0,53.196551,<NA>,S,None,18:19:03,2022-07-30 09:46:28,NaN


### Fake names (and others)

As one can see, the `Name` of the passengers is not quite so well synthesized. The reason is that the string type interpreter in MetaSynth is designed for `structured` strings (like room numbers such as `B1.09`, `B1.01` or `A1.08`) and not unstructured strings. However, MetaSynth supports the [faker](https://faker.readthedocs.io/en/master/index.html) package, which includes a lot of data types that it can fake. The columns using faker are not based on the real data at all so they do not disclose any info about the real data.

We fake names as follows:

In [14]:
# First, we create a specification dictionary for the variables
var_spec = {
    "PassengerId": {"unique": True}, 
    "Name": {"distribution": "faker.name"}
}

meta_dataset = MetaDataset.from_dataframe(df, spec=var_spec)
meta_dataset.synthesize(5)

,PassengerId,Name,Sex,Age,Parch,Fare,Cabin,Embarked,Birthday,Board time,Married since,all_NA
0,1,Joanne Lopez,male,73,0,34.490925,<NA>,S,None,13:45:56,2022-08-14 08:47:36,NaN
1,2,Cheryl Gomez,female,61,0,30.574295,<NA>,S,1914-06-17,14:05:30,2022-07-28 15:43:43,NaN
2,3,Jonathan Alvarado,female,46,0,113.777211,<NA>,S,1904-12-27,11:08:32,2022-07-18 05:20:29,NaN
3,4,Jennifer Frey,male,76,0,2.612483,<NA>,S,1938-07-16,11:19:09,2022-07-21 11:47:20,NaN
4,5,Christopher Patterson,female,14,0,3.687726,B`n1<X00,S,1936-02-25,15:53:07,2022-08-04 13:11:04,NaN


### Set distributions manually

Without user input, the distribution chosen for each variable is inferred by choosing the best fitting from available distributions for the variable type. However, we can also manually specify which distribution to fit, or we can even just fully specify how the variable should be generated.

In [15]:
from metasynth.distribution import DiscreteUniformDistribution

var_spec = {
    "PassengerId": {"unique": True}, 
    "Name": {"distribution": "faker.name"},
    "Fare": {"distribution": "LogNormalDistribution"}, # estimate / fit a log-normal distribution based on the data
    "Age": {"distribution": DiscreteUniformDistribution(20, 40)} # fully specify a distribution for age (uniform between 20 and 40)
}

meta_dataset = MetaDataset.from_dataframe(df, spec=var_spec)
meta_dataset.synthesize(5)

,PassengerId,Name,Sex,Age,Parch,Fare,Cabin,Embarked,Birthday,Board time,Married since,all_NA
0,1,Dr. Vickie Gibson,male,29,0,0.497739,<NA>,S,1938-05-16,12:50:20,2022-08-03 00:12:49,NaN
1,2,Charles Gonzalez,female,39,1,1.508213,<NA>,S,1934-10-17,15:06:27,2022-08-01 04:30:17,NaN
2,3,Jacob Macias,female,29,0,1.162552,<NA>,S,1914-02-22,14:34:01,2022-08-06 22:36:45,NaN
3,4,Sherry Zhang,male,39,0,1.262280,<NA>,S,1938-03-28,16:41:06,2022-08-04 15:40:31,NaN
4,5,Melissa Ford,male,24,0,0.225318,B{w+q+t!>`S_E,S,1909-02-05,14:28:35,2022-08-13 11:32:59,NaN


### Specifying the distribution of structured strings

For more or less structured strings, we can manually set the structure of the strings based on regular expressions. For example, we see that most Cabins are structured like [A-F] and then 2 or 3 digit numbers. We can include this as follows:

In [16]:
from metasynth.distribution import RegexDistribution

# To create a regex distribution, you need a list of tuples, where each tuple is an element.
# The first part of the tuple is a string representation of the regex, while the second is the proportion of the
# time the regex element is used.
cabin_distribution = RegexDistribution(r"[ABCDEF]\d{2,3}")  # Add the r so that it becomes a literal string.
# just for completeness: data generated from this distribution will always match the regex [ABCDEF]?(\d{2,3})?

var_spec = {
    "PassengerId": {"unique": True}, 
    "Name": {"distribution": "faker.name"},
    "Fare": {"distribution": "LogNormalDistribution"}, # estimate / fit a log-normal distribution based on the data
    "Age": {"distribution": DiscreteUniformDistribution(20, 40)}, # fully specify a distribution for age (uniform between 20 and 40)
    "Cabin": {"distribution": cabin_distribution}
}

meta_dataset = MetaDataset.from_dataframe(df, spec=var_spec)
meta_dataset.synthesize(10)

,PassengerId,Name,Sex,Age,Parch,Fare,Cabin,Embarked,Birthday,Board time,Married since,all_NA
0,1,Mark Taylor,male,29,0,0.573778,<NA>,S,1928-02-18,18:18:08,2022-08-07 06:25:08,NaN
1,2,Heather Smith,male,35,1,2.843856,<NA>,S,None,15:23:30,NaT,NaN
2,3,Martha Lopez,female,32,0,0.634045,<NA>,S,1940-04-29,11:20:59,2022-08-12 10:55:15,NaN
3,4,Samantha Franklin,male,30,2,0.488813,<NA>,S,1933-09-10,12:46:14,2022-08-01 04:54:01,NaN
4,5,Sean Fischer,male,34,0,0.632773,<NA>,S,None,11:58:47,2022-07-20 02:14:29,NaN
5,6,Travis Black,male,35,2,0.388725,<NA>,S,1912-11-11,13:38:29,2022-07-25 03:30:25,NaN
6,7,Christine Taylor,male,39,0,0.667790,<NA>,S,1907-06-18,18:39:32,2022-07-29 02:51:34,NaN
7,8,Abigail Hines,female,37,0,2.174496,<NA>,S,1908-10-04,12:01:39,2022-07-28 00:05:36,NaN
8,9,Taylor Orozco,female,38,0,0.709035,<NA>,Q,1914-06-18,11:08:22,2022-08-11 07:25:47,NaN
9,10,Rodney Williams,female,33,0,0.991394,C57,C,1923-07-31,13:08:03,2022-07-21 06:16:59,NaN
